In [ ]:
#test

# 必要なライブラリのインポート
from bs4 import BeautifulSoup
import requests

# スクレイピングするURLのリスト
urls = [
    "https://nf3.sakura.ne.jp/php/stat_disp/stat_disp.php?y=2022&leg=0&mon=0&tm=DB&vst=all",
    "https://nf3.sakura.ne.jp/php/stat_disp/stat_disp.php?y=0&leg=0&mon=0&tm=DB&vst=all"
]

# スクレイピングしたデータを保存するためのリスト
scraped_data = []